# First load os and sys so I can update the sys.path with new functions

In [1]:
import os
import sys

In [2]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [3]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [4]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

Using TensorFlow backend.


In [5]:
#function to scale our data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler()
def scaleColumns(df, cols_to_scale):
    for col in cols_to_scale:
        df[col] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df


In [6]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [7]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/1L/inclusive/wh1.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/1L/inclusive/whsm.csv",sep='\s+',engine='python')


In [11]:
signal['signal']

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
99970    1
99971    1
99972    1
99973    1
99974    1
99975    1
99976    1
99977    1
99978    1
99979    1
99980    1
99981    1
99982    1
99983    1
99984    1
99985    1
99986    1
99987    1
99988    1
99989    1
99990    1
99991    1
99992    1
99993    1
99994    1
99995    1
99996    1
99997    1
99998    1
99999    1
Name: signal, Length: 100000, dtype: int64

In [8]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [9]:
print(combined.keys())

Index(['ptb1', 'ptb2', 'ptl', 'missinget', 'pth', 'ptw', 'etah', 'phih',
       'deltarwl', 'mtW', 'mtvh', 'ptvh', 'delphibl', 'delphilmet', 'signal'],
      dtype='object')


In [10]:
combined['signal']

54588    1
92181    1
1204     0
84475    0
1560     1
13689    0
27221    1
37995    0
55958    0
89612    1
67722    1
8311     0
14670    1
31640    0
92703    0
89210    1
70448    1
64882    1
94444    1
12465    1
51410    0
22964    0
52603    0
13246    1
51597    0
88703    1
48154    1
22605    1
36071    0
15249    0
        ..
155      1
63969    0
98854    0
52181    0
72396    1
88108    1
8605     0
56775    0
6066     1
40942    0
10111    1
78406    1
98484    0
48623    1
9030     0
10905    1
8852     0
21496    1
82780    0
7178     0
92051    0
22327    1
43616    0
45306    1
65502    1
82186    0
47642    0
94807    0
41795    1
24219    1
Name: signal, Length: 200000, dtype: int64

In [ ]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            '1L':['ptb1', 'ptb2', 'ptl', 'missinget', 'pth', 'ptw', 'etah', 'phih',
                  'deltarwl', 'mtW', 'mtvh', 'ptvh', 'delphibl', 'delphilmet', 'signal']
}

In [ ]:
trainedModels={}

In [ ]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [ ]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    'dnn_WH_1L_cHW_0d03_batch_128':{'epochs':200,'batch_size':128,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[128],
                 'optimizer':'rmsprop', 'activation':'tanh'},
    'dnn_WH_1L_cHW_0d03_batch_256':{'epochs':200,'batch_size':256,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[128],
                 'optimizer':'rmsprop', 'activation':'tanh'},
    'dnn_WH_1L_cHW_0d03_batch_512':{'epochs':200,'batch_size':512,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[128],
                 'optimizer':'rmsprop', 'activation':'tanh'},
    'dnn_WH_1L_cHW_0d03_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[128],
                 'optimizer':'rmsprop', 'activation':'tanh'}    
        }

In [ ]:
lumi=30. #luminosity in /fb
expectedSignal=500*lumi 
expectedBkgd=200*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.1 #systematic for the asimov signficance

In [ ]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '1L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
        dnn = Dnn(mlData,'testPlots/mlPlots/'+varSetName+'/'+name)
        dnn.setup(hiddenLayers=config['hiddenLayers'], dropOut=config['dropOut'],
                  l2Regularization=config['l2Regularization'], optimizer=config['optimizer'],
                  activation=config['activation'],
                extraMetrics=[
                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                    ])
        dnn.fit(epochs=config['epochs'],batch_size=config['batch_size'],callbacks=[earlyStopping])
        
        dnn.explainPredictions()
        dnn.diagnostics(batchSize=config['batch_size'])
        dnn.makeHepPlots(expectedSignal,expectedBkgd,asimovSigLossSysts,makeHistograms=False)

        trainedModels[varSetName+'_'+name]=dnn